In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import sklearn
import yellowbrick
import re
import mglearn
import boto3
from s3 import get_file

In [2]:
s3 = boto3.resource('s3')
lyrics = get_file(s3,'s3ssp',download_file='NLP_Data/new_master_lyrics_audio_features.csv',rename_file='nlp.csv')

In [3]:
df = pd.read_csv(lyrics,sep='|',encoding='utf-8')
df_demo = df.copy().dropna()

In [4]:
df_sample = df_demo.sample(3000)

In [16]:
df_sample['new_lyrics'] = df_sample['lyrics'].apply(lambda x: x.replace("love", "").replace("chorus", ""))

In [6]:
def LDA_hyper_params(df):
    
    df_topic_rows = df.shape[0] #Dataframe Row count
    
    if(df_topic_rows in range(100,1000)):
        return {'max_features':100, 'n_components':3, 'n_words':10}
    
    elif(df_topic_rows in range(1001,4000)):
        return {'max_features':200, 'n_components' :5, 'n_words':10} 
        
    elif(df_topic_rows in range(4001,6000)):
        return {'max_features':10000, 'n_components' :25, 'n_words':10} 
    
    elif(df_topic_rows in range(6001,8000)):
        return {'max_features':400, 'n_components' :5, 'n_words':10} 
    
    elif(df_topic_rows in range(8001,10000)):
        return {'max_features':500, 'n_components' :5, 'n_words':10}
    
    elif(df_topic_rows >= 10000):
        return {'max_features':600, 'n_components' :5, 'n_words':10}
    else:
        return None

In [8]:
#Control Panel - Hyper Parameters

hp = LDA_hyper_params(df_sample)

#Count Vectorizer
max_features = 3000  #possibly a percentage of vect.vocabulary_
max_df = .5
min_df = 10



#LDA
n_components = 30  #10


#LDA Display
display_n_chunks = 5 #5
#n_components = hp['n_components']        #20
n_words = 10       #5


print(f'Max Features: {max_features} \nTopics: {n_components} \nWords: {n_words}')

Max Features: 3000 
Topics: 30 
Words: 10


In [10]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_df, max_df=max_df,     # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=max_features)             # max number of uniq words

data_vectorized = vectorizer.fit_transform(df_sample['lyrics'])



In [13]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=n_components, learning_method="online",
                                max_iter=25, random_state=0, doc_topic_prior=.06) #topic_word_prior=10000)

# Build LDA Model
lda_model.fit(data_vectorized)

# We build the model and transform the data in one step
# Computing transform takes some time,
# and we can save time by doing both at once

#document_topics = lda_model.fit_transform(docs)


#print(lda_model)  # Model attributes

# Log Likelyhood: Higher the better
#print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
#print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters



#print("lda.components_.shape: {}".format(lda.components_.shape))

# For each topic (a row in the components_), sort the features (ascending)
# Invert rows with [:, ::-1] to make sorting descending
#sorting = np.argsort(lda_model.components_, axis=1)#[:, ::-1]




# Get the feature names from the vectorizer
#feature_names = np.array(vectorizer.get_feature_names())

# Print out the 10 topics:
#mglearn.tools.print_topics(topics=range(n_components), feature_names=feature_names,
                           #sorting=sorting, topics_per_chunk=display_n_chunks, n_words=n_words)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=0.06,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=25, mean_change_tol=0.001,
             n_components=30, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [14]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

/Applications/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.253478  0.009654       1        1  15.357366
1      0.246190 -0.082384       2        1  10.772742
13     0.222824 -0.019406       3        1   7.867168
12     0.096566 -0.162012       4        1   6.494723
22    -0.011386  0.247677       5        1   5.861095
26     0.119130  0.139352       6        1   5.003521
20    -0.000781  0.206985       7        1   4.999987
8      0.116276  0.106848       8        1   3.916234
14     0.139838 -0.062305       9        1   3.873418
0      0.062225 -0.192871      10        1   3.744019
21     0.055013 -0.177932      11        1   3.567100
29     0.026134 -0.005532      12        1   2.574190
24    -0.035429 -0.006668      13        1   2.565941
11     0.093827  0.024890      14        1   2.502628
28     0.076711  0.007711      15        1   2.496881
2     -0.013794  0.183342      16        1   2.226071
17    -0.014391 -0.043206      17        1   1.802573
23    -0.058332 -0.032645      18        1   1.734598
10    -0.048904  0.023090      19        1   1.653186
3      0.022251  0.061008      20        1   1.585866
19    -0.038684 -0.022628      21        1   1.585523
27    -0.099597  0.113903      22        1   1.476206
4     -0.217242 -0.025472      23        1   1.440920
9     -0.122842  0.004148      24        1   1.169034
25    -0.159525 -0.073303      25        1   0.937949
15    -0.141906 -0.117865      26        1   0.881891
5     -0.158318 -0.038381      27        1   0.712505
6     -0.192459 -0.033943      28        1   0.574605
16    -0.089282 -0.017055      29        1   0.368250
7     -0.127587 -0.015000      30        1   0.253813, topic_info=     Category         Freq       Term        Total  loglift  logprob
950   Default  3452.000000       love  3452.000000  30.0000  30.0000
697   Default  1333.000000        gon  1333.000000  29.0000  29.0000
70    Default  1417.000000       baby  1417.000000  28.0000  28.0000
1821  Default  1401.000000       yeah  1401.000000  27.0000  27.0000
1789  Default   941.000000        win   941.000000  26.0000  26.0000
681   Default   954.000000       girl   954.000000  25.0000  25.0000
558   Default   863.000000       fall   863.000000  24.0000  24.0000
1644  Default  2436.000000       time  2436.000000  23.0000  23.0000
248   Default   646.000000     change   646.000000  22.0000  22.0000
1751  Default   881.000000        wan   881.000000  21.0000  21.0000
168   Default   553.000000        boy   553.000000  20.0000  20.0000
1078  Default  1067.000000      night  1067.000000  19.0000  19.0000
405   Default  1322.000000        day  1322.000000  18.0000  18.0000
903   Default  1135.000000      leave  1135.000000  17.0000  17.0000
288   Default   404.000000      close   404.000000  16.0000  16.0000
1097  Default   413.000000        ooh   413.000000  15.0000  15.0000
1748  Default   576.000000       walk   576.000000  14.0000  14.0000
583   Default  1938.000000       feel  1938.000000  13.0000  13.0000
1107  Default   379.000000       pain   379.000000  12.0000  12.0000
1478  Default   492.000000       soul   492.000000  11.0000  11.0000
916   Default  1556.000000       life  1556.000000  10.0000  10.0000
1782  Default   298.000000      white   298.000000   9.0000   9.0000
760   Default  1181.000000      heart  1181.000000   8.0000   8.0000
942   Default   747.000000       lose   747.000000   7.0000   7.0000
1263  Default   441.000000   remember   441.000000   6.0000   6.0000
772   Default   479.000000        hey   479.000000   5.0000   5.0000
185   Default   428.000000      bring   428.000000   4.0000   4.0000
646   Default   470.000000     friend   470.000000   3.0000   3.0000
393   Default   339.000000      dance   339.000000   2.0000   2.0000
271   Default   721.000000     chorus   721.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
690   Topic30     0.0

In [17]:
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='mmds')

/Applications/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.000088 -0.103234       1        1  15.357366
1     -0.106042 -0.244684       2        1  10.772742
13    -0.109881 -0.091796       3        1   7.867168
12     0.031676 -0.270244       4        1   6.494723
22     0.418561 -0.162165       5        1   5.861095
26     0.230493 -0.240006       6        1   5.003521
20     0.338900  0.163546       7        1   4.999987
8      0.170565 -0.415090       8        1   3.916234
14    -0.278143 -0.234625       9        1   3.873418
0     -0.316277 -0.367998      10        1   3.744019
21    -0.337146  0.004963      11        1   3.567100
29    -0.459482 -0.034662      12        1   2.574190
24    -0.088503  0.175694      13        1   2.565941
11    -0.128447 -0.441719      14        1   2.502628
28     0.133108  0.369967      15        1   2.496881
2     -0.202536  0.297806      16        1   2.226071
17    -0.353550  0.215966      17        1   1.802573
23    -0.459754 -0.195044      18        1   1.734598
10     0.036389 -0.498214      19        1   1.653186
3      0.343088 -0.361083      20        1   1.585866
19     0.320212 -0.075909      21        1   1.585523
27     0.313905  0.396847      22        1   1.476206
4     -0.487143  0.208098      23        1   1.440920
9      0.043912  0.475808      24        1   1.169034
25    -0.268904  0.431626      25        1   0.937949
15    -0.091885  0.482147      26        1   0.881891
5      0.474937  0.162230      27        1   0.712505
6      0.497602  0.001491      28        1   0.574605
16     0.124290  0.113722      29        1   0.368250
7      0.210144  0.236561      30        1   0.253813, topic_info=     Category         Freq       Term        Total  loglift  logprob
950   Default  3452.000000       love  3452.000000  30.0000  30.0000
697   Default  1333.000000        gon  1333.000000  29.0000  29.0000
70    Default  1417.000000       baby  1417.000000  28.0000  28.0000
1821  Default  1401.000000       yeah  1401.000000  27.0000  27.0000
1789  Default   941.000000        win   941.000000  26.0000  26.0000
681   Default   954.000000       girl   954.000000  25.0000  25.0000
558   Default   863.000000       fall   863.000000  24.0000  24.0000
1644  Default  2436.000000       time  2436.000000  23.0000  23.0000
248   Default   646.000000     change   646.000000  22.0000  22.0000
1751  Default   881.000000        wan   881.000000  21.0000  21.0000
168   Default   553.000000        boy   553.000000  20.0000  20.0000
1078  Default  1067.000000      night  1067.000000  19.0000  19.0000
405   Default  1322.000000        day  1322.000000  18.0000  18.0000
903   Default  1135.000000      leave  1135.000000  17.0000  17.0000
288   Default   404.000000      close   404.000000  16.0000  16.0000
1097  Default   413.000000        ooh   413.000000  15.0000  15.0000
1748  Default   576.000000       walk   576.000000  14.0000  14.0000
583   Default  1938.000000       feel  1938.000000  13.0000  13.0000
1107  Default   379.000000       pain   379.000000  12.0000  12.0000
1478  Default   492.000000       soul   492.000000  11.0000  11.0000
916   Default  1556.000000       life  1556.000000  10.0000  10.0000
1782  Default   298.000000      white   298.000000   9.0000   9.0000
760   Default  1181.000000      heart  1181.000000   8.0000   8.0000
942   Default   747.000000       lose   747.000000   7.0000   7.0000
1263  Default   441.000000   remember   441.000000   6.0000   6.0000
772   Default   479.000000        hey   479.000000   5.0000   5.0000
185   Default   428.000000      bring   428.000000   4.0000   4.0000
646   Default   470.000000     friend   470.000000   3.0000   3.0000
393   Default   339.000000      dance   339.000000   2.0000   2.0000
271   Default   721.000000     chorus   721.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
690   Topic30     0.0

In [18]:
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

/Applications/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
18      54.507607  127.030785       1        1  15.357366
1      -44.619602 -160.003143       2        1  10.772742
13      -0.018205   78.830040       3        1   7.867168
12     141.186401   93.297066       4        1   6.494723
22    -106.599098  -11.611065       5        1   5.861095
26     167.212311   24.231842       6        1   5.003521
20    -187.396179   89.711769       7        1   4.999987
8      -65.583145  199.517441       8        1   3.916234
14     -51.959568   36.320305       9        1   3.873418
0       31.763491  205.888535      10        1   3.744019
21      34.592941 -181.333603      11        1   3.567100
29    -123.420799 -142.334427      12        1   2.574190
24      46.663624 -109.269287      13        1   2.565941
11      72.330162   63.731846      14        1   2.502628
28    -129.803192  148.091415      15        1   2.496881
2     -166.033569  -69.518776      16        1   2.226071
17     100.513702  -60.965122      17        1   1.802573
23      93.821297    4.462958      18        1   1.734598
10     -19.247412  -91.421059      19        1   1.653186
3     -120.223442   56.105766      20        1   1.585866
19      19.029884   19.182055      21        1   1.585523
27      32.730648  -40.932827      22        1   1.476206
4      -15.451523  144.871140      23        1   1.440920
9      -35.084663  -25.132591      24        1   1.169034
25     -70.069168  103.723686      25        1   0.937949
15     175.031555  -54.475872      26        1   0.881891
5      116.865295  168.285965      27        1   0.712505
6     -185.250565    6.191635      28        1   0.574605
16     -85.990074  -76.855133      29        1   0.368250
7      123.113579 -135.557388      30        1   0.253813, topic_info=     Category         Freq       Term        Total  loglift  logprob
950   Default  3452.000000       love  3452.000000  30.0000  30.0000
697   Default  1333.000000        gon  1333.000000  29.0000  29.0000
70    Default  1417.000000       baby  1417.000000  28.0000  28.0000
1821  Default  1401.000000       yeah  1401.000000  27.0000  27.0000
1789  Default   941.000000        win   941.000000  26.0000  26.0000
681   Default   954.000000       girl   954.000000  25.0000  25.0000
558   Default   863.000000       fall   863.000000  24.0000  24.0000
1644  Default  2436.000000       time  2436.000000  23.0000  23.0000
248   Default   646.000000     change   646.000000  22.0000  22.0000
1751  Default   881.000000        wan   881.000000  21.0000  21.0000
168   Default   553.000000        boy   553.000000  20.0000  20.0000
1078  Default  1067.000000      night  1067.000000  19.0000  19.0000
405   Default  1322.000000        day  1322.000000  18.0000  18.0000
903   Default  1135.000000      leave  1135.000000  17.0000  17.0000
288   Default   404.000000      close   404.000000  16.0000  16.0000
1097  Default   413.000000        ooh   413.000000  15.0000  15.0000
1748  Default   576.000000       walk   576.000000  14.0000  14.0000
583   Default  1938.000000       feel  1938.000000  13.0000  13.0000
1107  Default   379.000000       pain   379.000000  12.0000  12.0000
1478  Default   492.000000       soul   492.000000  11.0000  11.0000
916   Default  1556.000000       life  1556.000000  10.0000  10.0000
1782  Default   298.000000      white   298.000000   9.0000   9.0000
760   Default  1181.000000      heart  1181.000000   8.0000   8.0000
942   Default   747.000000       lose   747.000000   7.0000   7.0000
1263  Default   441.000000   remember   441.000000   6.0000   6.0000
772   Default   479.000000        hey   479.000000   5.0000   5.0000
185   Default   428.000000      bring   428.000000   4.0000   4.0000
646   Default   470.000000     friend   470.000000   3.0000   3.0000
393   Default   339.000000      dance   339.000000   2.0000   2.0000
271   Default   721.000000     

In [57]:
# Create Document - Topic Matrix

lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]


# index names
docnames = ["Doc" + str(i) for i in range(len(df_sample['lyrics']))]



# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.array(lda_output), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)


df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .6 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .6 else 400
    return 'font-weight: {weight}'.format(weight=weight)


df_document_topic = df_document_topic[df_document_topic['Topic1']>=.65]
# Apply Style
#df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)

for col in df_document_topic.columns:
    topic_length=df_document_topic[df_document_topic[col]>=.65].nlargest(n=20, columns=col)
    if len(topic_length)>=10:
        #print(df_document_topic[col].nlargest(n=20))
        print(col)

Topic1
dominant_topic


In [45]:
type(df_document_topic)

pandas.core.series.Series

In [44]:
df_sample['lyrics'].iloc[1296]

'command warrior ready attack destroy robot destroy inhuman form life destroy destroy destroy'

'strike fast hard mercy vermin christ prophet lie disciple seek hunt break spirit crush heart death set pain charge ride flee steel draw blood suffer field sacrifice wipe burn field feed wolf offspring annihilate master torment soul rape whore carry cross burn burn alive send soul deathqueen hall land cold burning flame send land famine despair eternally starve freeze'

lot
set
strange
age
fun
hero
death
final
circle
air
water
march
sigh
tide
tie
people
wrap
bass
guitar
box
break
gate
form
church
machine
worship
